### Задание
Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats 
import matplotlib.pyplot as plt  

In [2]:
data = pd.read_csv('adult.csv')
data.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [3]:
incomes = data['income']
incomes.unique()

array(['<=50K', '>50K'], dtype=object)

In [4]:
ad_income = data[['age', 'educational-num','income']]
ad_income

,age,educational-num,income
0,25,7,<=50K
1,38,9,<=50K
2,28,12,>50K
3,44,10,>50K
4,18,10,<=50K
...,...,...,...
48837,27,12,<=50K
48838,40,9,>50K
48839,58,9,<=50K
48840,22,9,<=50K


In [5]:
ad_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   educational-num  48842 non-null  int64 
 2   income           48842 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [6]:
ad_income.describe()

,age,educational-num
count,48842.000000,48842.000000
mean,38.643585,10.078089
std,13.710510,2.570973
min,17.000000,1.000000
25%,28.000000,9.000000
50%,37.000000,10.000000
75%,48.000000,12.000000
max,90.000000,16.000000


###### построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
X = ad_income.apply(LabelEncoder().fit_transform)

In [10]:
del X['income']
X.head()

,age,educational-num
0,8,6
1,21,8
2,11,11
3,27,9
4,1,9


In [11]:
le.fit(ad_income['income'])

LabelEncoder()

In [12]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [13]:
le.transform( ['<=50K', '>50K', '<=50K'])

array([0, 1, 0])

In [14]:
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=100000))

In [15]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=100000))])

In [16]:
y = pd.Series (le.transform(incomes))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [19]:
predictions[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1])

In [21]:
model.predict_proba(X_test)

array([[0.74223464, 0.25776536],
       [0.91677506, 0.08322494],
       [0.54383939, 0.45616061],
       ...,
       [0.72209156, 0.27790844],
       [0.96846172, 0.03153828],
       [0.44677329, 0.55322671]])

In [22]:
model.score(X_train, y_train) 

0.7825864407647224

In [23]:
model.score(X_test,y_test)

0.7851366567714197

##### SVC

In [24]:
# используем пайплайны
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [25]:
clf.score(X_train, y_train) 

0.7915184398433701

In [26]:
clf.score(X_test, y_test)

0.7950660251816972

In [27]:
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [28]:
svc.score(X_train, y_train)

0.7945128349499655

In [29]:
svc.score(X_test, y_test)

0.7941447435766199